___
<a href='https://cafe.naver.com/jmhonglab'><p style="text-align:center;"><img src='https://lh3.googleusercontent.com/lY3ySXooSmwsq5r-mRi7uiypbo0Vez6pmNoQxMFhl9fmZJkRHu5lO2vo7se_0YOzgmDyJif9fi4_z0o3ZFdwd8NVSWG6Ea80uWaf3pOHpR4GHGDV7kaFeuHR3yAjIJjDgfXMxsvw=w2400'  class="center" width="50%" height="50%"/></p></a>
___
<center><em>Content Copyright by HongLab, Inc.</em></center>

# 제너레이터 기반 코루틴

### [PEP 342 – Coroutines via Enhanced Generators](https://peps.python.org/pep-0342/)

비교) Native coroutine (AsyncIO)

Co + Routine = (하나가 끝나면 번갈아가면서 일하는 것이 아니라) 동시에 함께 일하는 루틴  
제너레이터와 프로그램의 흐름을 주고 받으면서 동시에 일하는 것 처럼 진행  
```yield``` vs ```send()```

In [40]:
def what_to_say():
    str = "안녕?"
    while True:
        name = (yield str)
        str = name + ", 안녕?"

coro = what_to_say()

# yield를 만나기 위해 처음에는 next() 필수 (send() 사용 X)
print(next(coro)) # 안녕?

# 제너레이터가 출력할 내용을 만들어 주면
# 메인에서 실제로 출력
print(coro.send("보노보노")) # 보노보노, 안녕?
print(coro.send("차차")) # 차차, 안녕?
print(coro.send("텔레토비")) # 텔레토비, 안녕?
# 계속 진행 ...

안녕?
보노보노, 안녕?
차차, 안녕?
텔레토비, 안녕?


send()대신 close()로 코루틴 종료

In [37]:
def what_to_say():
    str = "안녕?"
    while True:
        try:
            name = (yield str)
            str = name + ", 안녕?"
        except GeneratorExit:
            print("종료: 이제 그만!")
            return # 제너레이터 종료

coro = what_to_say()

print(next(coro)) # yield를 만나기 위해 처음에는 next() 필수 (send() 사용 X)

# 제너레이터가 출력할 내용을 만들어 주면
# 메인에서 실제로 출력
print(coro.send("보노보노"))
print(coro.send("차차"))
print(coro.send("텔레토비"))
# ... 여러 가지 일들 진행

# 메인에서 종료시킬 수 있음
coro.close()

# 제너레이터 종료 후 send()는 StopIteration 발생
print(coro.send("텔레토비"))

안녕?
보노보노, 안녕?
차차, 안녕?
텔레토비, 안녕?
종료: 이제 그만!


StopIteration: 


병렬성: 여러 사람들이 여러 컴퓨터로 각자 일하는 상황  
동시성: 여러 사람들이 컴퓨터 하나로 번갈아가면서 일하는 상황  

In [35]:
# 직렬 예제

import time

def my_task(task_name, duration):
    start_time = time.time()
    while time.time() - start_time < duration:
        continue # IO-Bound task
    print("Finished ", task_name)

my_task("TaskA", 1.0)
my_task("TaskB", 1.0)

Finished  TaskA
Finished  TaskB


In [42]:
# 동시성 예제
import time

def my_task(task_name, start_time, duration):
    current_time = start_time
    while current_time - start_time < duration:
        # IO-Bound task
        current_time = (yield "Working")
        print(current_time, task_name)

    print("Finished ", task_name)
    yield None

start_time = time.time()

coro1 = my_task("TaskA", start_time, 1.0)
coro2 = my_task("TaskB", start_time, 0.5)

next(coro1)
next(coro2)

task_list = [coro1, coro2]

while task_list:
    current_time = time.time() # 현재 시간을 한 번 측정해서 여러 코루틴들에게 배포
    for t in task_list.copy():
        if not t.send(current_time):
            task_list.remove(t)

    time.sleep(0.1)



1649902936.0145235 TaskA
1649902936.0145235 TaskB
1649902936.1198673 TaskA
1649902936.1198673 TaskB
1649902936.2305875 TaskA
1649902936.2305875 TaskB
1649902936.3409004 TaskA
1649902936.3409004 TaskB
1649902936.452897 TaskA
1649902936.452897 TaskB
1649902936.5648978 TaskA
1649902936.5648978 TaskB
Finished  TaskB
1649902936.6761487 TaskA
1649902936.786158 TaskA
1649902936.8961349 TaskA
1649902937.0074856 TaskA
1649902937.1174226 TaskA
Finished  TaskA
